## Imports

In [7]:
import pandas as pd

## Read CSV

In [8]:
# Read csv
data = pd.read_csv('data_processed.csv')
print("Shape: " + str(data.shape))

Shape: (26447, 23)


## Gaze and Head Positions Adjusts

In [9]:
timeBlock = 10 # minutes
time_adjust = '10s'

In [10]:
data['datetime'] = pd.to_datetime(data['datetime'])
data['time_block'] = (data['datetime'] - data['datetime'].min()).dt.total_seconds() // (60 * timeBlock)
persons = data['person'].unique()

# Adjust angles for each person in blocks of 10 minutes
for person in persons:
    mask_person = data['person'] == person
    
    for block in data.loc[mask_person, 'time_block'].unique():
        mask_block = (data['time_block'] == block) & mask_person # Intersection between person and block
        data_person_block = data[mask_block]
        
        data.loc[mask_block, 'gaze_pitch_mean'] = data_person_block['gaze_pitch'].mean()
        data.loc[mask_block, 'gaze_yaw_mean'] = data_person_block['gaze_yaw'].mean()
        data.loc[mask_block, 'head_pitch_mean'] = data_person_block['head_pitch'].mean()
        data.loc[mask_block, 'head_yaw_mean'] = data_person_block['head_yaw'].mean()
        data.loc[mask_block, 'head_roll_mean'] = data_person_block['head_roll'].mean()

In [11]:
data.set_index('datetime', inplace=True)
result = []

# Adjust gaze and head angles for each person in blocks of 10 minutes each 10 seconds
for (person, time_block), group in data.groupby(['person', 'time_block']):

    group['gaze_pitch_adjusted'] = group['gaze_pitch'].resample(time_adjust).transform('mean') - group['gaze_pitch_mean'].iloc[0]
    group['gaze_yaw_adjusted'] = group['gaze_yaw'].resample(time_adjust).transform('mean') - group['gaze_yaw_mean'].iloc[0]
    group['head_pitch_adjusted'] = group['head_pitch'].resample(time_adjust).transform('mean') - group['head_pitch_mean'].iloc[0]
    group['head_yaw_adjusted'] = group['head_yaw'].resample(time_adjust).transform('mean') - group['head_yaw_mean'].iloc[0]
    group['head_roll_adjusted'] = group['head_roll'].resample(time_adjust).transform('mean') - group['head_roll_mean'].iloc[0]
    
    result.append(group)

data = pd.concat(result).sort_index()
data.reset_index(inplace=True)

## Variation in facial movement for each person

In [12]:
person1 = data['person'].unique()

for i in person1:
    data_person_i = data[data['person'] == i]

    face_width = data_person_i['face_bbox_x2'] - data_person_i['face_bbox_x1']
    average_pixel_per_cm = (face_width/12).mean() # 12 centimeters is the value by default for a face's width

    data.loc[data_person_i.index, 'head_movement_x_abs'] = (data_person_i['face_center_x'].diff().bfill() / average_pixel_per_cm).abs()
    data.loc[data_person_i.index, 'head_movement_y_abs'] = (data_person_i['face_center_y'].diff().bfill() / average_pixel_per_cm).abs()
    data.loc[data_person_i.index, 'head_movement_z_abs'] = (face_width.diff().bfill() / average_pixel_per_cm).abs()

### Group in intervals

In [13]:
person1 = data['person'].unique()

for i in person1:
    data_person_i = data[data['person'] == i]
    columns_to_group = ['head_movement_x_abs', 'head_movement_y_abs', 'head_movement_z_abs']

    for column in columns_to_group:
        data_person_grouped = data_person_i.groupby(pd.Grouper(key='datetime', freq=time_adjust))[column].mean().reset_index()
        data_person_grouped.columns = ['datetime', f'{column}_{time_adjust}']

        for j in data_person_grouped.values:
            start_time = j[0]
            time_delta = pd.to_timedelta(time_adjust)
            end_time = start_time + time_delta
            mask = (data['datetime'] >= start_time) & (data['datetime'] < end_time) & (data['person'] == i)
            data.loc[mask, f'{column}_{time_adjust}'] = j[1]

## Difference between Gaze and Head Position

In [14]:
data['head_gaze_diff_yaw_abs'] = (data['head_yaw_adjusted'] - data['gaze_yaw_adjusted']).abs()
data['head_gaze_diff_pitch_abs'] = (data['head_pitch_adjusted'] - data['gaze_pitch_adjusted']).abs()

## Blink Rate

In [ ]:
person1 = data['person'].unique()
data['blink'] = 0

for i in person1:
    data_person_i = data[data['person'] == i]
    prev_left_eye = "open"
    prev_right_eye = "open"

    for idx, row in data_person_i.iterrows():
        left_eye = row['left_eye_state']
        right_eye = row['right_eye_state']

        if prev_left_eye == "open" and prev_right_eye == "open" and (left_eye != "open" or right_eye != "open"):
            data.at[idx, 'blink'] = 1

        prev_left_eye = left_eye
        prev_right_eye = right_eye


        # Fazer groupby(PERSON) intervalos de 1 segundo e dentro desse nº de leituras se tiver 1 leitura em que o olho esq e olho dir fechado consideramos um blink
        # só é considerado olho fechado se confidence > 0.6 (if)          

In [20]:
data['blink_rate'] = 0

for i in person1:
    data_person_i = data[data['person'] == i].copy()
    data_person_i = data_person_i.set_index('datetime')
    data_person_i['blink_rate'] = data_person_i['blink'].rolling('60s').sum().fillna(0)
    data.loc[data['person'] == i, 'blink_rate'] = data_person_i['blink_rate'].values

In [23]:
for i in person1:
    data_person_i = data[data['person'] == i]
    print(data_person_i['blink_rate'].value_counts())
    print("................................")


blink_rate
22    386
24    319
10    318
25    312
23    276
21    272
26    249
20    239
18    236
15    203
9     199
11    168
12    161
28    152
16    152
19    145
35    140
42    136
2     130
17    126
13    124
14    123
34    121
27    113
8     112
39    111
40    105
45    103
38    101
41    100
43     96
29     92
31     89
1      85
32     84
44     80
6      76
33     75
37     74
5      72
46     67
30     66
36     63
4      47
7      32
47     22
0      21
3      12
Name: count, dtype: int64
................................
blink_rate
28    299
27    292
37    286
38    278
36    267
34    264
30    259
26    248
31    247
35    241
29    211
32    207
39    203
33    192
45    181
44    178
49    167
23    165
43    159
48    153
41    151
42    146
4     144
25    144
47    140
24    135
50    131
51    123
40    111
22    107
46    106
15    103
21     93
52     69
16     63
11     54
13     50
14     46
20     42
18     38
12     33
19     23
54     20
17     18

## Engagement

### Normalize Data

In [ ]:
dataNormalized = pd.DataFrame()
# Difference between the head and gaze angles
dataNormalized['head_gaze_diff_yaw_abs'] = data['head_gaze_diff_yaw_abs']/data['head_gaze_diff_yaw_abs'].max()
dataNormalized['head_gaze_diff_pitch_abs'] = data['head_gaze_diff_pitch_abs']/data['head_gaze_diff_pitch_abs'].max()

# Normalize the values of head angles
dataNormalized['head_pitch_abs'] = data['head_pitch_adjusted'].abs()/data['head_pitch_adjusted'].abs().max()
dataNormalized['head_yaw_abs'] = data['head_yaw_adjusted'].abs()/data['head_yaw_adjusted'].abs().max()
dataNormalized['head_roll_abs'] = data['head_roll_adjusted'].abs()/data['head_roll_adjusted'].abs().max()

# Normalize the values of gaze angles
dataNormalized['gaze_pitch_abs'] = data['gaze_pitch_adjusted'].abs()/data['gaze_pitch_adjusted'].abs().max()
dataNormalized['gaze_yaw_abs'] = data['gaze_yaw_adjusted'].abs()/data['gaze_yaw_adjusted'].abs().max()

# Normalize the movement of the head
dataNormalized['head_movement_x'] = data['head_movement_x_abs']/data['head_movement_x_abs'].max()
dataNormalized['head_movement_x_abs_10s'] = data['head_movement_x_abs_10s']/data['head_movement_x_abs_10s'].max()
dataNormalized['head_movement_y'] = data['head_movement_y_abs']/data['head_movement_y_abs'].max()
dataNormalized['head_movement_y_abs_10s'] = data['head_movement_y_abs_10s']/data['head_movement_y_abs_10s'].max()
dataNormalized['head_movement_z'] = data['head_movement_z_abs']/data['head_movement_z_abs'].max()
dataNormalized['head_movement_z_abs_10s'] = data['head_movement_z_abs_10s']/data['head_movement_z_abs_10s'].max()

dataNormalized['blink_rate'] = data['blink_rate']/data['blink_rate'].max()

### Engagement Formula

In [ ]:
gaze_yaw = dataNormalized['gaze_yaw_abs']
gaze_pitch = dataNormalized['gaze_pitch_abs']
gaze = (1 - (0.5 * gaze_pitch + 0.5 * gaze_yaw))

head_yaw = dataNormalized['head_yaw_abs']
head_pitch = dataNormalized['head_pitch_abs']
head_roll = dataNormalized['head_roll_abs']
head = (1 - (0.333 * head_pitch + 0.333 * head_yaw + 0.333 * head_roll))


head_gaze_diff_yaw = dataNormalized['head_gaze_diff_yaw_abs']
head_gaze_diff_pitch = dataNormalized['head_gaze_diff_pitch_abs']
head_gaze_diff = (1 - (0.5 * head_gaze_diff_pitch + 0.5 * head_gaze_diff_yaw))

head_movement_x_10s = dataNormalized['head_movement_x_abs_10s']
head_movement_y_10s = dataNormalized['head_movement_y_abs_10s']
head_movement_z_10s = dataNormalized['head_movement_z_abs_10s']
head_movement_10s = (1 - (0.333 * head_movement_x_10s + 0.333 * head_movement_y_10s + 0.333 * head_movement_z_10s))

blink_rate = (1 - dataNormalized['blink_rate'])



# Formula to calculate engagement
data['engagement'] = 0.20 * head + 0.20 * gaze + 0.20 * head_gaze_diff + 0.20 * head_movement_10s + 0.20 * blink_rate

## Create Final Dataset

In [ ]:
data_final = data[['datetime', 'person', 'engagement','facial_expression']]
data_final.to_csv('data_final.csv', index=False)